<a href="https://colab.research.google.com/github/nanoNNsk/superaiss5_hackathon/blob/main/superaiss5_iot_sleep_cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import dataset

In [ ]:
!pip install kaggle

In [ ]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 82.0 MB/s eta 0:00:00
   ━━━

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sibthinonkruesri","key":"1c75217df587ae1a22950d9c66384616"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c io-t-sleep-stage-classification-version-2

100% 2.07G/2.08G [00:12<00:00, 275MB/s]
100% 2.08G/2.08G [00:12<00:00, 174MB/s]


In [ ]:
!unzip /content/io-t-sleep-stage-classification-version-2.zip

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
  inflating: test_segment/test_segment/test003/test003_00692.csv  
  inflating: test_segment/test_segment/test003/test003_00693.csv  
  inflating: test_segment/test_segment/test003/test003_00694.csv  
  inflating: test_segment/test_segment/test003/test003_00695.csv  
  inflating: test_segment/test_segment/test003/test003_00696.csv  
  inflating: test_segment/test_segment/test003/test003_00697.csv  
  inflating: test_segment/test_segment/test003/test003_00698.csv  
  inflating: test_segment/test_segment/test003/test003_00699.csv  
  inflating: test_segment/test_segment/test003/test003_00700.csv  
  inflating: test_segment/test_segment/test004/test004_00000.csv  
  inflating: test_segment/test_segment/test004/test004_00001.csv  
  inflating: test_segment/test_segment/test004/test004_00002.csv  
  inflating: test_segment/test_segment/test004/test004_00003.csv  
  inflating: test_segment/test_segment/test004/test004_00004.csv  
  inf

In [ ]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 7.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import dask.dataframe as dd
import os

train_folder_path = "/content/train/train"
train_data = []
for file in sorted(os.listdir(train_folder_path)):
    file_path = os.path.join(train_folder_path,file)
    train_file = dd.read_csv(file_path)
    train_file = train_file.compute()
    train_data.append(train_file)
train_data = pd.concat(train_data,ignore_index=True)

In [ ]:
train_data.head()

,BVP,ACC_X,ACC_Y,ACC_Z,TEMP,EDA,HR,IBI,Sleep_Stage
0,25.325870,-21.809247,-60.302750,4.940839,31.722653,0.064595,72.015570,1.050338,W
1,20.021505,-19.437787,-60.565345,7.408788,31.722647,0.064523,72.015802,1.050338,W
2,16.314478,-21.624667,-61.142561,4.105717,31.722735,0.064659,72.017417,1.050338,W
3,9.324392,-21.761314,-61.985822,3.972967,31.722564,0.064440,72.013801,1.050338,W
4,-1.014338,-19.055301,-59.934137,9.097628,31.722790,0.065397,72.018920,1.050338,W


# prepare dataset

In [ ]:
train_data['Sleep_Stage'].unique()

<ArrowStringArray>
['W', 'N', 'R']
Length: 3, dtype: string

In [ ]:
train_data['Sleep_Stage'].value_counts()

,count
Sleep_Stage,
N,20856000
W,7692960
R,3358080


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31907040 entries, 0 to 31907039
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   BVP          float64
 1   ACC_X        float64
 2   ACC_Y        float64
 3   ACC_Z        float64
 4   TEMP         float64
 5   EDA          float64
 6   HR           float64
 7   IBI          float64
 8   Sleep_Stage  string 
dtypes: float64(8), string(1)
memory usage: 2.1 GB


In [ ]:
from sklearn.preprocessing import LabelEncoder
Sleep_encoder = LabelEncoder()
train_data['Sleep_Stage'] = Sleep_encoder.fit_transform(train_data['Sleep_Stage'])

In [ ]:
train_data.head()

,BVP,ACC_X,ACC_Y,ACC_Z,TEMP,EDA,HR,IBI,Sleep_Stage
0,25.325870,-21.809247,-60.302750,4.940839,31.722653,0.064595,72.015570,1.050338,2
1,20.021505,-19.437787,-60.565345,7.408788,31.722647,0.064523,72.015802,1.050338,2
2,16.314478,-21.624667,-61.142561,4.105717,31.722735,0.064659,72.017417,1.050338,2
3,9.324392,-21.761314,-61.985822,3.972967,31.722564,0.064440,72.013801,1.050338,2
4,-1.014338,-19.055301,-59.934137,9.097628,31.722790,0.065397,72.018920,1.050338,2


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31907040 entries, 0 to 31907039
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   BVP          float64
 1   ACC_X        float64
 2   ACC_Y        float64
 3   ACC_Z        float64
 4   TEMP         float64
 5   EDA          float64
 6   HR           float64
 7   IBI          float64
 8   Sleep_Stage  int64  
dtypes: float64(8), int64(1)
memory usage: 2.1 GB


In [ ]:
train_data['Sleep_Stage'].unique()

array([2, 0, 1])

In [ ]:
df_stage_0 = train_data[train_data['Sleep_Stage'] == 0].sample(n=100000, random_state=1)
df_stage_1 = train_data[train_data['Sleep_Stage'] == 1].sample(n=100000, random_state=1)
df_stage_2 = train_data[train_data['Sleep_Stage'] == 2].sample(n=100000, random_state=1)
final_train_data = pd.concat([df_stage_0, df_stage_1, df_stage_2])
final_train_data = final_train_data.sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
final_train_data['Sleep_Stage'].value_counts()

,count
Sleep_Stage,
0,20856000
2,7692960
1,3358080


In [ ]:
final_train_data.info()

# train model

In [ ]:
from pycaret.classification import *
classification_setup = setup(data=train_data, target='Sleep_Stage', session_id=42)

In [ ]:
best_model = compare_models()

In [ ]:
trained_model = create_model(best_model)

In [ ]:
evaluate_model(trained_model)

In [ ]:
tuned_model = tune_model(trained_model)

In [ ]:
final_model = finalize_model(tuned_model)

# test

In [ ]:
test_file_path = '/kaggle/input/io-t-sleep-stage-classification-version-2/test_segment/test_segment/test001/test001_00000.csv'
df=pd.read_csv(test_file_path)
df

In [ ]:
prediction = predict_model(final_model,df)
prediction

In [ ]:
most_common_label = prediction['prediction_label'].value_counts().idxmax()
most_common_label

In [ ]:
all_test_folder = '/kaggle/input/io-t-sleep-stage-classification-version-2/test_segment/test_segment'

In [ ]:
result = []
for folder in sorted(os.listdir(all_test_folder)):
    folder_path = os.path.join(all_test_folder, folder)
    if os.path.isdir(folder_path):
        for file in sorted(os.listdir(folder_path)):
            file_path = os.path.join(folder_path, file)
            data = pd.read_csv(file_path)
            prediction = predict_model(final_model,data)
            most_common_label = prediction['prediction_label'].value_counts().idxmax()
            result.append({'file_name': os.path.splitext(file)[0],\
                            'most_common_label': most_common_label})

In [ ]:
type(result)

In [ ]:
result[:10]

In [ ]:
copy_result = result[:]

In [ ]:
copy_result[:10]

In [ ]:
item in copy_result:
    item['most_common_label'] = Sleep_encoder.inverse_transform([item['most_common_label']])[0]

In [ ]:
copy_result[:10]

# submission

In [ ]:
submission = pd.read_csv('/kaggle/input/io-t-sleep-stage-classification-version-2/sample_submission.csv')

In [ ]:
submission

In [ ]:
result_df = pd.DataFrame(copy_result)
merged_df = submission.merge(result_df, how='left', left_on='id', right_on='file_name')
for index, row in merged_df.iterrows():
    if row['id'] == row['file_name']:
        submission.at[index, 'labels'] = row['most_common_label']

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)